# First Steps with TensorFlow

## 1. Constant tensors and variables

In [1]:
import tensorflow as tf

# All one or all-zeros tensors

x1 = tf.ones(shape = (2, 1))  # Equivalent to np.ones(shape = (2, 1))
x2 = tf.zeros(shape = (3, 2))

print(x1)
print(x2)


tf.Tensor(
[[1.]
 [1.]], shape=(2, 1), dtype=float32)
tf.Tensor(
[[0. 0.]
 [0. 0.]
 [0. 0.]], shape=(3, 2), dtype=float32)


## 2. Random tensors

In [7]:
# random numbers draw from a normal distribution

x3 = tf.random.normal(shape=(3, 4))

print(x3)

tf.Tensor(
[[ 0.69138986 -1.1380928  -0.10478156 -0.9882285 ]
 [-0.75249517 -0.91564167  2.2383628  -0.38101903]
 [-1.9399822   1.0147933  -0.29315448 -0.04191277]], shape=(3, 4), dtype=float32)


## 3. TensorFlow tensors are not assignable, unlike Numpy arrays

In [ ]:
# x3[0, 0] = 0

TypeError: 'tensorflow.python.framework.ops.EagerTensor' object does not support item assignment

## 4. Create a TensorFlow Variable

In [12]:
# create a tensor variable v1
v1 = tf.Variable(initial_value=tf.random.normal(shape=(3, 2)))

print(v1)


# Assign values to v1
v1.assign(tf.ones((3, 2)))
print(v1)

# Assign a value to v1
v1[0, 0].assign(0)
print(v1)

<tf.Variable 'Variable:0' shape=(3, 2) dtype=float32, numpy=
array([[ 1.9938629 , -1.7025366 ],
       [ 1.3097914 , -0.8987432 ],
       [-0.41046202,  0.42448917]], dtype=float32)>
<tf.Variable 'Variable:0' shape=(3, 2) dtype=float32, numpy=
array([[1., 1.],
       [1., 1.],
       [1., 1.]], dtype=float32)>
<tf.Variable 'Variable:0' shape=(3, 2) dtype=float32, numpy=
array([[0., 1.],
       [1., 1.],
       [1., 1.]], dtype=float32)>


## 5. Math operations in TensorFlow

In [17]:
my_list = [[1., 2.], [3., 4.]]
a = tf.constant(my_list)
b = tf.square(a)
c = tf.sqrt(a)
d = b + c
e = tf.matmul(a, b) # matrix multiplication
f = e * d

print([a, b, c, d, e, f])

[<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[1., 2.],
       [3., 4.]], dtype=float32)>, <tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[ 1.,  4.],
       [ 9., 16.]], dtype=float32)>, <tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[1.       , 1.4142135],
       [1.7320508, 2.       ]], dtype=float32)>, <tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[ 2.       ,  5.4142137],
       [10.732051 , 18.       ]], dtype=float32)>, <tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[19., 36.],
       [39., 76.]], dtype=float32)>, <tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[  38.    ,  194.9117],
       [ 418.55  , 1368.    ]], dtype=float32)>]


## 6. Use GradientTape API to retrieve the gradient of differentiable expression

## Example 1: Gradient of a Simple Function
Let's start with a basic mathematical function, $y=x^2$. We want to find the gradient of y with respect to x, which we know is $y = 2x$.

In [2]:
# Define a variable 'x'. GradientTape automatically watches tf.Variable.
x = tf.Variable(3.0)

# Start the GradientTape context
with tf.GradientTape() as tape:
  # Define the function y = x^2
  y = tf.square(x)

# Calculate the gradient of y with respect to x
dy_dx = tape.gradient(y, x)

# .numpy() converts a TensorFlow tensor into a NumPy array.
print(f"The gradient of y with respect to x at x = 3.0 is: {dy_dx.numpy()}")
# Expected output: 6.0 (since 2 * 3.0 = 6.0)

The gradient of y with respect to x at x = 3.0 is: 6.0


## How to Compute **secondary derivative** in TensorFlow ⚙️
You use two GradientTape blocks, one nested inside the other.

1. The **inner tape** records the operations to compute the first gradient.

2. The **outer tape** records the operations of the inner tape, including the calculation of the first gradient.

In [22]:
# Use a tf.Variable, as tapes automatically watch them.
x = tf.Variable(2.0)

# Nest two GradientTape contexts
with tf.GradientTape() as outer_tape:
    with tf.GradientTape() as inner_tape:
        # Define the function inside the inner tape
        y = x**3
    
    # Calculate the 1st order gradient using the inner tape.
    # The outer tape records this operation.
    first_gradient = inner_tape.gradient(y, x)

# Calculate the 2nd order gradient using the outer tape.
# This is the gradient of the first gradient.
second_gradient = outer_tape.gradient(first_gradient, x)

print(f"Original function value at x=2.0: y = {y.numpy()}")
print(f"First-order gradient at x=2.0 (3 * 2^2): {first_gradient.numpy()}")
print(f"Second-order gradient at x=2.0 (6 * 2): {second_gradient.numpy()}")

Original function value at x=2.0: y = 8.0
First-order gradient at x=2.0 (3 * 2^2): 12.0
Second-order gradient at x=2.0 (6 * 2): 12.0


By default, a GradientTape releases its resources as soon as `tape.gradient()` is called. If you need to compute multiple gradients from the same computation, you must create a persistent tape.

In [21]:
import tensorflow as tf

x = tf.Variable(2.0)

# Create a persistent tape
with tf.GradientTape(persistent=True) as tape:
  y = x**2
  z = x**3

# Calculate the first gradient
dy_dx = tape.gradient(y, x)
print(f"dy/dx at x=2.0 is: {dy_dx.numpy()}") # Should be 2*2 = 4.0

# Calculate the second gradient from the same tape
dz_dx = tape.gradient(z, x)
print(f"dz/dx at x=2.0 is: {dz_dx.numpy()}") # Should be 3*(2^2) = 12.0

# Manually delete the tape when you're done with it to free up memory
del tape

dy/dx at x=2.0 is: 4.0
dz/dx at x=2.0 is: 12.0
